# Tabulky s None

S tabulkami se pracuje pomocí knihovny Pandas. Nejsnazší je tvořit tabulky z
matice nebo po sloupcích. Přitom musí mít každý sloupec stejný počet dat. Potom
jsou všechny sloupce stejně dlouhé. Toto se však nedá zařídit vždy, někdy je
nutno do tabulky přidat sloupec, který je kratší. Řešením je přidat do tabulky
sloupec s libovolnými hodnotami a poté nahradit ta data, která máme k dispozici.
Vhodnou výplní nedefinovaných dat jsou nuly nebo hodnoty `None` nebo `np.nan`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

N = 11
df = pd.DataFrame()
df["a"] = np.linspace(1,20,N)

# založíme prázdný sloupec a potom jeho začátek přepíšeme daty
df["b"] = np.nan
data = np.array([2,3,4,5])
df.loc[:len(data)-1,"b"] = data

# jiná varianta je nejprve doplnit data a potom zakládat sloupec
data = np.array([15,12,9])
doplnena_data = np.append(data,np.full(N-len(data),np.nan))
df["c"] = doplnena_data

df.plot()
df

Ukázkou je rovnice lovu, kdy v pro některé počáteční podmínky je populace
přelovená a dochází k její destrukci. Řešení se nemusí zastavit po dosažení
cílového času, ale při poklesu velikosti populace na nulu. Pokud se snažíme
zapsat dat do jedné tabulky, musíme neobsazená místa zaplnit nedefinovanými hodnotami.

In [ ]:
### Příprava funkcí a parametrů
pocatecni_podminky = np.round(np.arange(0.1,1.3,0.02),2)  # počáteční podmínka nebo podmínky
meze = [0,15]  # interval, na kterém hledáme řešení
N = 100 # počet dělících bodů

def model(t, N, r=1,K=1,h=0.2):
    return  r*N*(1-N/K) - h

def destrukce_populace(t,x,r=1,K=1,h=0.15):  # Pokud x klesne na nulu, zastavíme výpočet
    return x
destrukce_populace.terminal = True

### Řešení modelu
t=np.linspace(*meze, N)  # definiční obor, v těchto bodech budeme hledat řešení
df = pd.DataFrame()
df["t"] = t

for pocatecni_podminka in pocatecni_podminky:
    reseni = solve_ivp(
                       model,
                       meze,
                       [pocatecni_podminka],
                       t_eval=t,
                       events=destrukce_populace,
                       )
    data = reseni.y[0]
    if len(data)==N:
        df[pocatecni_podminka] = data
    else:
        df[pocatecni_podminka] = None
        df.loc[:len(data)-1,pocatecni_podminka] = data
        # jina varianta je nejprve doplnit data a potom zakladat sloupec
        # doplnena_data = np.append(data,np.full(N-len(data),np.nan))
        # df[pocatecni_podminka] = doplnena_data

### Vizualizace řešení

ax = df.plot(x="t",lw=0.5,legend=False,color="C0")
ax.set(
    ylim = (0,None),
    title = "Řešení diferenciální rovnice",
    xlabel=r"$t$",
    ylabel=r"$x$",
);

Začátek a konec tabulky s jedním předčasně končícím řešením.

In [ ]:
df.loc[:,[0.24,0.5,0.8]].head(n=10)

In [ ]:
df.loc[:,[0.24,0.5,0.8]].tail(n=10)